In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook

In [31]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [32]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [33]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [34]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(X)
    hidden2 = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(hidden1)
    logits = tf.keras.layers.Dense(n_outputs)(hidden2)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [35]:
batch_size = 50

finish = False
no_progress_counter = 0
minimum_error = 1
epoch = 0

with tf.Session() as sess:
    init.run()
    while finish == False:
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        error = loss.eval(feed_dict={X: X_valid, y: y_valid})
        if minimum_error > error:
            minimum_error = error
            no_progress_counter = 0
        else:
            no_progress_counter += 1
            if no_progress_counter == 50: finish = True
        print(epoch, 'Train loss:', error, 'Progress counter:', no_progress_counter)
        epoch += 1
    save_path = saver.save(sess, "./my_model_finalv1.ckpt")

0 Train loss: 0.35616454 Progress counter: 0
1 Train loss: 0.27835962 Progress counter: 0
2 Train loss: 0.24131997 Progress counter: 0
3 Train loss: 0.21570364 Progress counter: 0
4 Train loss: 0.19629258 Progress counter: 0
5 Train loss: 0.1849023 Progress counter: 0
6 Train loss: 0.16717534 Progress counter: 0
7 Train loss: 0.15514867 Progress counter: 0
8 Train loss: 0.14740945 Progress counter: 0
9 Train loss: 0.14121787 Progress counter: 0
10 Train loss: 0.13235195 Progress counter: 0
11 Train loss: 0.12601897 Progress counter: 0
12 Train loss: 0.123594224 Progress counter: 0
13 Train loss: 0.11641362 Progress counter: 0
14 Train loss: 0.1121094 Progress counter: 0
15 Train loss: 0.10632257 Progress counter: 0
16 Train loss: 0.10339074 Progress counter: 0
17 Train loss: 0.103190884 Progress counter: 0
18 Train loss: 0.0965985 Progress counter: 0
19 Train loss: 0.09443557 Progress counter: 0
20 Train loss: 0.092901275 Progress counter: 0
21 Train loss: 0.09137751 Progress counter: 

In [36]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [37]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
